## Real Time Testing

In [2]:
! py -3.9 -m pip install -r "../requirements.txt"

Looking in indexes: https://download.pytorch.org/whl/cu121


ERROR: Could not find a version that satisfies the requirement transformers==4.35.2 (from versions: none)
ERROR: No matching distribution found for transformers==4.35.2


In [3]:
from collections import deque
from concurrent.futures import ThreadPoolExecutor
from time import time
from cvzone import FPS

import cv2
import numpy as np
from itertools import chain
from time import time

from cvzone.HandTrackingModule import HandDetector
from cvzone.FaceDetectionModule import FaceDetector
from cvzone.PoseModule import PoseDetector

In [4]:
## 2. Feature Extraction (Hand+Face+Pose Detection)
# Flatten a 2d np array into 1d array
def flatten2dList(arr, dataType=int):
    return np.fromiter(chain.from_iterable(arr), dataType)

# Get the largest absolute value in an np array
def getAbsLargestVal(arr):
    return np.max(np.abs(arr))

# Offset and normalize the landmark list
# Returns a 1d numpy array
def preprocess_landmarks(landmark_list):    
    landmark_list = np.array(landmark_list, dtype=float)
    origin = landmark_list[0]
    
    # Offset every point with respect to the first point
    # Convert to 1D-array
    new_landmark_list = (landmark_list - origin).ravel()
    
    # Get highest absolute value
    largest_value = getAbsLargestVal(new_landmark_list)
    
    # Normalization
    if largest_value != 0:
        return new_landmark_list / largest_value
    return new_landmark_list

# Offset and normalize a BBOX list (BBOX = Bounding Box, used in face and hand detection)
# Returns a 1d numpy array
def preprocess_bbox(bbox, frameSize):
    bbox = np.array(bbox, dtype=float)
    # Convert 3rd and 4th element into coordinates instead of width/height
    bbox[2] = bbox[0] + bbox[2]
    bbox[3] = bbox[1] + bbox[3]

    # Normalize against frame size
    bbox[0] /= frameSize[0]
    bbox[1] /= frameSize[1]
    bbox[2] /= frameSize[0]
    bbox[3] /= frameSize[1]

    return bbox

# Normalize a center vertex (a list of 2 elements)
# Returns a 1d numpy array
def preprocess_center(center, frameSize):
    center = np.array(center, dtype=float)
    center[0] /= frameSize[0]
    center[1] /= frameSize[1]
    return center

# Preprocess (Offset and normalize) the body's landmark list, bbox and center
def preprocess_body_part(bodyPart, frameSize):
    bodyPart['lmList'] = preprocess_landmarks(bodyPart['lmList'])
    bodyPart['bbox'] = preprocess_bbox(bodyPart['bbox'], frameSize)
    bodyPart['center'] = preprocess_center(bodyPart['center'], frameSize)
    return bodyPart

# Function to generate empty/placeholder data for a hand 
# Used when a hand is not detected in frame
def generate_empty_hand(type):
    return {
        'lmList': np.zeros(21 * 3, dtype=int), 
        'bbox': np.zeros(4, dtype=float), 
        'center': np.zeros(2, dtype=float), 
        'type': type
    }

# Select the best matching face, aka the one with the best score (clarity)
# and closest to the center of the screen
# Since the Neural network will be design to only accept one face
def select_best_matching_face(faces, frameSize):
    if not faces or len(faces) == 0:
        return False
    elif len(faces) == 1:
        return faces[0]
    
    def difference(a, b):
        return ((a[0] - b[0])**2) + ((a[1] - b[1])**2)
    
    frameCenter = (frameSize[0] / 2, frameSize[1] / 2)

    best_score = faces[0]
    best_center = faces[0]
    center_diff = difference(faces[0]['center'], frameCenter)

    for each in faces:
        if difference(each['center'], frameCenter) < center_diff:
            best_center = each
        if each['score'][0] > best_score['score'][0]:
            best_score = each
    
    if best_center['score'][0] > 0.5:
        return best_center
    return best_score

# Flatten everything
def flattenDetectionResult(obj):
    # return np.fromiter(chain.from_iterable([obj['lmList'], obj['bbox'], obj['center']]), float)
    return np.concatenate([obj['lmList'], obj['bbox'], obj['center']])

In [5]:
# Detects hands
# convert them into normalized landmark/keypoint coordinates in a 1D-array, 
# and also returns the frame with the landmark connections drawn onto it

# Improved/Parallelised version
def staticFeatureExtraction(handDetector, frame, draw=True):
    def detectHands(handDetector, frame, frameSize, draw):
        results = None
        # Hand Detection
        if (draw):
            results, frame = handDetector.findHands(frame, draw=draw)
        else:
            results = handDetector.findHands(frame, draw=draw)

        if not results:
            results = [generate_empty_hand('Left'), generate_empty_hand('Right')]
        elif len(results) == 1:
            if (results[0]['type'] == 'Left'):
                results[0] = preprocess_body_part(results[0], frameSize)
                results.append(generate_empty_hand('Right'))
            else:
                results[0] = preprocess_body_part(results[0], frameSize)
                results.insert(0, generate_empty_hand('Left'))                         
        else:
            results[0] = preprocess_body_part(results[0], frameSize)
            results[1] = preprocess_body_part(results[1], frameSize)
        return results

    frameSize = (frame.shape[1], frame.shape[0])
    with ThreadPoolExecutor() as executor:
        t1 = executor.submit(detectHands, handDetector, frame, frameSize, draw)
        
        # Convert results into 1D-array
        detectionResults = flatten2dList([
            flattenDetectionResult(t1.result()[0]), 
            flattenDetectionResult(t1.result()[1]), 
        ], dataType=float)

        return detectionResults, frame

In [6]:
import keras
from files_io import readActionLabels

# Load the model from the H5 file
model = keras.models.load_model('../static_recognition/models/static_model.h5')

# Print the summary before loading
model.summary()

static_labels = readActionLabels()
static_labels

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 136, 32)           128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 68, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 66, 64)            6208      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 33, 64)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 31, 128)           24704     
                                                                 
 flatten_1 (Flatten)         (None, 3968)             

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'one',
 'two',
 'three',
 'four',
 'five',
 'six',
 'seven',
 'eight',
 'nine',
 'ten']

In [7]:
cam = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# Detectors
handDetector = HandDetector(detectionCon=0.5, maxHands=2)
faceDetector = FaceDetector(minDetectionCon=0.5)
poseDetector = PoseDetector(detectionCon=0.5)
fpsReader = FPS()

timeStats = []

try:
    predictionHistory = deque()
    detectionThreshold = 0.99999    
    predictionCooldown = 0.5
    appendCooldown = 1.0
    # the first time append should eliminate predictionCooldown
    lastAppendTime = time() + appendCooldown
    lastDetectTime = time() + predictionCooldown

    while True:
        startTime = time()

        # Read from camera
        success, frame = cam.read()
        if not success:
            raise Exception("No Frames Read")
        frame = cv2.flip(frame, 1)

        # Hand Detection
        inputValue, frame = staticFeatureExtraction(
            handDetector, frame)
        
        
        detectionResults = np.expand_dims(
            inputValue, axis=0) # reshape detection result
        
        if time() <= lastDetectTime + predictionCooldown:
            pass
        else:
            predictionResults = model.predict(
                x=detectionResults,
                verbose=0,
                use_multiprocessing=True,
                workers=4
            )[0]
            # lastDetectTime = time()
            
            # Get predicted character and accuracy
            predCharacter = static_labels[np.argmax(predictionResults)]
            predAccuracy = predictionResults[np.argmax(predictionResults)]
        
            cv2.putText(frame, (predCharacter + ":"+ str(predAccuracy)), (15, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)          
            
            if predAccuracy >= detectionThreshold:            
                # If predictionHistory is not empty
                # If predCharacter is the same as the last appended character
                # Check if 0.5 seconds have passed since the last append
                if predictionHistory and predCharacter == predictionHistory[-1] and time() <= lastAppendTime + appendCooldown:
                    # Do nothing, don't append
                    pass
                else:
                    predictionHistory.append(predCharacter)
                    # Reset the timestamp when a new character is detected
                    lastAppendTime = time()

                if len(predictionHistory) > 5:
                        predictionHistory.popleft()  
                        
        cv2.putText(frame, str(predictionHistory), (15, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
        

        fps, frame = fpsReader.update(frame, pos=(
            950, 80), color=(0, 255, 0),scale=5, thickness=5)
        
        # Show resulting frame
        cv2.imshow("Sign Language Recognition Prototype", frame)

        timeStats.append(time() - startTime)

        keyPressed = cv2.waitKey(1)
        # Stop Program when pressed 'Esc'
        if keyPressed == 27:
            raise Exception("Finished")
        elif keyPressed == ord('c'):
            predictionHistory.clear()
            lastAppendTime = time() + predictionCooldown


except Exception as e:
    cam.release()
    cv2.destroyAllWindows()
    print(e)

cam.release()
cv2.destroyAllWindows()

Finished


In [8]:
timeStats[10:]

[0.047748565673828125,
 0.058339595794677734,
 0.019025802612304688,
 0.04598212242126465,
 0.04737567901611328,
 0.0465540885925293,
 0.0480504035949707,
 0.043146371841430664,
 0.03509044647216797,
 0.05872941017150879,
 0.2620401382446289,
 0.06104636192321777,
 0.06340980529785156,
 0.06405901908874512,
 0.0657966136932373,
 0.06472539901733398,
 0.06070733070373535,
 0.0648188591003418,
 0.06511950492858887,
 0.06206035614013672,
 0.0652010440826416,
 0.06306648254394531,
 0.06459832191467285,
 0.06256222724914551,
 0.06413078308105469,
 0.06305456161499023,
 0.0630497932434082,
 0.06182694435119629,
 0.06682801246643066,
 0.06959366798400879,
 0.06639313697814941,
 0.06435942649841309,
 0.06614232063293457,
 0.06458592414855957,
 0.06491303443908691,
 0.06561064720153809,
 0.06668663024902344,
 0.06510615348815918,
 0.06854581832885742,
 0.06398534774780273,
 0.06757712364196777,
 0.06415176391601562,
 0.06457662582397461,
 0.06407284736633301,
 0.06669759750366211,
 0.0655906200